In [82]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch/minsearch.py

In [83]:
# !wget https://rawgithubusercontent.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json

In [84]:
import minsearch

In [85]:
import json

In [86]:
with open('../data/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [87]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [88]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [89]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [90]:
index.fit(documents)

In [91]:
from openai import OpenAI
import os

In [92]:
client = OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"],
)

In [93]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}
    results = index.search(
        query = query,
        # filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results=5
    )
    return results

In [94]:
search_results = search('how do i run kafka?')

In [95]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTIONS based
    on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    context = ""
    
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [96]:
print(build_prompt('how do i run kafka?', search_results))

You're a course teaching assistant. Answer the QUESTIONS based
    on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: how do i run kafka?
    
    CONTEXT:
    section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
source 

In [97]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
        ],
    model="openai/gpt-4o-mini",
    temperature=1,
    max_tokens=4096,
    top_p=1
)
    return response.choices[0].message.content

In [99]:
query = 'what else  do i need to do other than finishing the project to get a certificate?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [100]:
answer

'To get a certificate, you need to finish at least 2 out of 3 projects and review 3 peers’ projects by the deadline, in addition to finishing your project.'

In [101]:
search_results

[{'text': 'Yes, if you finish at least 2 out of 3 projects and review 3 peers’ Projects by the deadline, you will get a certificate. This is what it looks like: link. There’s also a version without a robot: link.',
  'section': 'General course-related questions',
  'question': 'Will I get a certificate?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'There is only ONE project for this Zoomcamp. You do not need to submit or create two projects. There are simply TWO chances to pass the course. You can use the Second Attempt if you a) fail the first attempt b) do not have the time due to other engagements such as holiday or sickness etc. to enter your project into the first attempt.',
  'section': 'Project',
  'question': 'Project 1 & Project 2',
  'course': 'data-engineering-zoomcamp'},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) a

In [102]:
query = 'can i enroll in the course if it already strted?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [103]:
rag(query)

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.'

In [104]:
from elasticsearch import Elasticsearch

In [105]:
es_client = Elasticsearch('http://elasticsearch:9200')

In [106]:
index_settings = {
    "settings":{
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings":{
        "properties":{
            "text":{"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
            
        }
    }

}   
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

In [107]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [108]:
from tqdm.auto import tqdm

In [109]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [110]:
if es_client.indices.exists(index=index_name):
    print(f"✅ Index '{index_name}' exists!")
else:
    print(f"❌ Index '{index_name}' does NOT exist.")


✅ Index 'course-questions' exists!


In [111]:
count = es_client.count(index=index_name)['count']
print(f"There are {count} documents in the index '{index_name}'.")

There are 1896 documents in the index 'course-questions'.


In [112]:
query = 'how can i install kafka?'

In [113]:
def elastic_search(query):
    search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "data-engineering-zoomcamp"
                        }
                    }
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [114]:
elastic_search(query)

[{'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
  'section': 'Module 6: streaming with kafka',
  'question': 'Confluent Kafka: Where can I find schema registry URL?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
  'section': 'Module 6: streaming with kafka',
  'question': 'Confluent Kafka: Where can I find schema registry URL?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Alternatively, you can switch to in-file storage with:',
  'section': 'Workshop 1 - dlthub',
  'question': 'How can I use DuckDB In-Memory database with dlt ?',
  'course': 'data-engineer

In [115]:
search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "data-engineering-zoomcamp"
                        }
                    }
                }
            }
        }

In [116]:
response = es_client.search(index=index_name, body=search_query)

In [117]:
response

ObjectApiResponse({'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 520, 'relation': 'eq'}, 'max_score': 29.817202, 'hits': [{'_index': 'course-questions', '_id': 'Af8BZ5kBaSgjd0X3v4Hr', '_score': 29.817202, '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it', 'section': 'Module 6: streaming with kafka', 'question': 'Confluent Kafka: Where can I find schema registry URL?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'tf8GZ5kBaSgjd0X3PYQd', '_score': 29.817202, '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Crede

In [118]:
response['hits']

{'total': {'value': 520, 'relation': 'eq'},
 'max_score': 29.817202,
 'hits': [{'_index': 'course-questions',
   '_id': 'Af8BZ5kBaSgjd0X3v4Hr',
   '_score': 29.817202,
   '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
    'section': 'Module 6: streaming with kafka',
    'question': 'Confluent Kafka: Where can I find schema registry URL?',
    'course': 'data-engineering-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'tf8GZ5kBaSgjd0X3PYQd',
   '_score': 29.817202,
   '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
    'section': 'Module 6: streaming with kafka',
    'question': 'C

In [119]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'Af8BZ5kBaSgjd0X3v4Hr',
  '_score': 29.817202,
  '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
   'section': 'Module 6: streaming with kafka',
   'question': 'Confluent Kafka: Where can I find schema registry URL?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'tf8GZ5kBaSgjd0X3PYQd',
  '_score': 29.817202,
  '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
   'section': 'Module 6: streaming with kafka',
   'question': 'Confluent Kafka: Where can I find schema registry URL?',
   'course': 'data-engineering-zoo

In [120]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [121]:
result_docs

[{'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
  'section': 'Module 6: streaming with kafka',
  'question': 'Confluent Kafka: Where can I find schema registry URL?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
  'section': 'Module 6: streaming with kafka',
  'question': 'Confluent Kafka: Where can I find schema registry URL?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Alternatively, you can switch to in-file storage with:',
  'section': 'Workshop 1 - dlthub',
  'question': 'How can I use DuckDB In-Memory database with dlt ?',
  'course': 'data-engineer

In [122]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [123]:
search_results = elastic_search(query)

In [124]:
prompt = build_prompt(query, search_results)
print(prompt)

You're a course teaching assistant. Answer the QUESTIONS based
    on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: how can i install kafka?
    
    CONTEXT:
    section: Module 6: streaming with kafka
question: Confluent Kafka: Where can I find schema registry URL?
answer: In Confluent Cloud:
Environment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”
And create credentials from Credentials section below it

section: Module 6: streaming with kafka
question: Confluent Kafka: Where can I find schema registry URL?
answer: In Confluent Cloud:
Environment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”
And create credentials from Credentials section below it

section: Workshop 1 - d

In [125]:
query = 'can i enroll in the course after it started'
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."